## Data Trainig

In [ ]:
from pathlib import Path
Path("src").mkdir(exist_ok=True)

In [ ]:
import digitalhub as dh

In [ ]:
PROJECT = "project-cml-darts-ci"
project = dh.get_or_create_project(PROJECT)

In [ ]:
%%writefile "src/train-model.py"


from digitalhub_runtime_python import handler

import pandas as pd
import numpy as np

from darts import TimeSeries
from darts.datasets import AirPassengersDataset
from darts.models import NBEATSModel
from darts.metrics import mape, smape, mae

from zipfile import ZipFile

@handler()
def train_model(project):
    series = AirPassengersDataset().load()
    train, val = series[:-36], series[-36:]

    model = NBEATSModel(
        input_chunk_length=24,
        output_chunk_length=12,
        n_epochs=200,
        random_state=0
    )
    model.fit(train)
    pred = model.predict(n=36)

    model.save("predictor_model.pt")
    with ZipFile("predictor_model.pt.zip", "w") as z:
        z.write("predictor_model.pt")
        z.write("predictor_model.pt.ckpt")
    metrics = {
        "mape": mape(series, pred),
        "smape": smape(series, pred),
        "mae": mae(series, pred)
    }

    model = project.log_model(
        name="darts_model",
        kind="model",
        source="predictor_model.pt.zip",
        algorithm="darts.models.NBEATSModel",
        framework="darts"
    )
    model.log_metrics(metrics)
    return model

In [ ]:
train_fn = project.new_function(name="train-darts",
                                kind="python",
                                python_version="PYTHON3_10",
                                code_src="src/train-model.py",
                                handler="train_model")

In [ ]:
train_build = train_fn.run("build",
                           instructions=["pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu","pip3 install darts patsy scikit-learn"],
                           wait=True)

Wait until build job is completed. One can check the status of 'Run' from console or by typing the following command

In [ ]:
train_build.refresh().status.state

Once completed, proceed to run the training job.

In [ ]:
train_run = train_fn.run("job", wait=True)

Wait for job to be completed. One can check the status of 'Run' from console or by typing the following command

In [ ]:
train_run.refresh().status.state

Once completed, proceed with serve run.

## Serve

In [ ]:
%%writefile "src/serve_darts_model.py"

from darts.models import NBEATSModel
from zipfile import ZipFile
from darts import TimeSeries
import json
import pandas as pd

def init(context):
    model_name = "darts_model"

    model = context.project.get_model(model_name)
    path = model.download()
    local_path_model = "extracted_model/"

    with ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(local_path_model)

    input_chunk_length = 24
    output_chunk_length = 12
    name_model_local = local_path_model +"predictor_model.pt"
    mm = NBEATSModel(
            input_chunk_length,
            output_chunk_length
    ).load(name_model_local)

    setattr(context, "model", mm)

def serve(context, event):

    if isinstance(event.body, bytes):
        body = json.loads(event.body)
    else:
        body = event.body
    context.logger.info(f"Received event: {body}")
    inference_input = body["inference_input"]

    pdf = pd.DataFrame(inference_input)
    pdf['date'] = pd.to_datetime(pdf['date'], unit='ms')

    ts = TimeSeries.from_dataframe(
        pdf,
        time_col="date",
        value_cols="value"
    )

    output_chunk_length = 12
    result = context.model.predict(n=output_chunk_length*2, series=ts)
    # Convert the result to a pandas DataFrame, reset the index, and convert to a list
    jsonstr = result.pd_dataframe().reset_index().to_json(orient='records')
    return json.loads(jsonstr)

In [ ]:
func = project.new_function(name="serve_darts_model",
                            kind="python",
                            python_version="PYTHON3_10",
                            code_src="src/serve_darts_model.py",
                            handler="serve",
                            init_function="init")

In [ ]:
run_build_model_serve = func.run("build",
                                 instructions=["pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu","pip3 install darts patsy scikit-learn"],
                                 wait=True)

Wait for job to be completed. One can check the status of 'Run' from console or by typing the following command

In [ ]:
run_build_model_serve.refresh().status.state

Once completed, run the serve job.

In [ ]:
serve_run = func.run("serve",
                     labels=["service"],
                     wait=True)

Once 'Service' object available, proceed by making call to deployed service.

In [ ]:
%pip install darts

It takes a while to install darts library. Then we prepare the inference set.

In [ ]:
import json
from datetime import datetime
from darts.datasets import AirPassengersDataset

series = AirPassengersDataset().load()
val = series[-24:]
json_value = json.loads(val.to_json())

data = map(lambda x, y: {"value": x[0], "date": datetime.timestamp(datetime.strptime(y, "%Y-%m-%dT%H:%M:%S.%f"))*1000}, json_value["data"], json_value["index"])
inputs = {"inference_input": list(data)}

In [ ]:
serve_run.invoke(json=inputs).json()

## Workflow Pipeline

In [ ]:
%%writefile "src/cml_darts_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler():
    with pipeline_context() as pc:
        trainer = pc.step(name="train-model", function="train-darts", action="job")

In [ ]:
workflow = project.new_workflow(name="pipeline_cml_darts",
                                kind="kfp",
                                code_src="src/cml_darts_pipeline.py",
                                handler ="myhandler")

In [ ]:
wf_run = workflow.run("build", wait=True)

In [ ]:
wf_run = workflow.run("pipeline", wait=True)